# Análise da utilização da cota para o exercício da atividade Parlamentar

## Cota Parlamentar

Instituída pelo Ato da Mesa 43/2009, a Cota para o Exercício da Atividade Parlamentar unificou a verba indenizatória (que vigorava desde 2001), a cota de passagens aéreas e a cota postal-telefônica. Visa ao custeio de despesas típicas do exercício do mandato parlamentar.

O valor mensal da Cota Parlamentar é utilizado pelo deputado por meio de reembolso; por meio de requisição de serviço postal, no caso de sua utilização na agência dos Correios credenciada pela Câmara dos Deputados; ou por meio de débito automático no valor da Cota, no caso de compra de bilhete aéreo realizada nas companhias aéreas credenciadas. Nesses dois últimos tipos de gasto, não há emissão individual de nota fiscal, uma vez que o contrato é assinado pela Casa.

O valor base varia de acordo com o estado e pode ir de R\\$ 30.788,66 (DF) a R\\$ 45.612,53 (RR), além do salário do deputado, que é em média de R\\$ 30 mil. Como o nome sugere, a cota é um valor destinado a auxiliar parlamentares no exercício do seu mandato, com usos diversos, que vão desde alimentação até o fretamento de aeronaves.

![Cota](https://cdn-images-1.medium.com/max/800/1*vuekZ9tOz8WGY99ZJqWH1Q.jpeg)

* http://www2.camara.leg.br/comunicacao/assessoria-de-imprensa/cota-parlamentar
* https://medium.com/data-science-brigade/precisamos-falar-sobre-a-cota-parlamentar-c58a73392148


### Operação Serenata de Amor

Um projeto aberto que usa ciência de dados - as mesmas tecnologias utilizadas por gigantes como Google, Facebook e Netflix - com a finalidade de fiscalizar gastos públicos e compartilhar as informações de forma acessível a qualquer pessoa.

* https://serenata.ai/

### Links uteis

* Acesso ao site da camara para buscar reenbolsos: http://www.camara.gov.br/cota-parlamentar

* Dicionário de dados oficial: http://www2.camara.leg.br/transparencia/cota-para-exercicio-da-atividade-parlamentar/explicacoes-sobre-o-formato-dos-arquivos-xml
```
'txNomeParlamentar': 'congressperson_name',
'idecadastro': 'congressperson_id',
'nuCarteiraParlamentar': 'congressperson_document',
'nuLegislatura': 'term',
'sgUF': 'state',
'sgPartido': 'party',
'codLegislatura': 'term_id',
'numSubCota': 'subquota_number',
'txtDescricao': 'subquota_description',
'numEspecificacaoSubCota': 'subquota_group_id',
'txtDescricaoEspecificacao': 'subquota_group_description',
'txtFornecedor': 'supplier',
'txtCNPJCPF': 'cnpj_cpf',
'txtNumero': 'document_number',
'indTipoDocumento': 'document_type',
'datEmissao': 'issue_date',
'vlrDocumento': 'document_value',
'vlrGlosa': 'remark_value',
'vlrLiquido': 'net_value',
'numMes': 'month',
'numAno': 'year',
'numParcela': 'installment',
'txtPassageiro': 'passenger',
'txtTrecho': 'leg_of_the_trip',
'numLote': 'batch_number',
'numRessarcimento': 'reimbursement_number',
'vlrRestituicao': 'reimbursement_value',
'nuDeputadoId': 'applicant_id',
'ideDocumento': 'document_id',
```


## Baixando Dados

In [ ]:
!wget https://storage.googleapis.com/ms_geral/2018-01-05-reimbursements.csv

In [ ]:
!ls

## Começando a Análise

### Leitura do Dataset

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 500)

In [ ]:
df = pd.read_csv('./2018-01-05-reimbursements.csv')

df.info()

In [ ]:
# Exibir a sumarização dos dados, ótimo para fazer uma primeira análise das estatísticas descritivas  média, min, max ..
df.describe()

### Agrupamentos - Total de Reembolso do Parlamentar por Ano

Fazer agrupamentos por variáveis "temporais" é interessante para criarmos uma timeline de valores

Para isso podemos criar funções de agregações customizadas para cada tipo de variável diferente.

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.aggregate.html


In [ ]:
agg_fun ={
    'total_net_value': 'sum'
}
year_sum = df.groupby('year', as_index=False).agg(agg_fun)
year_sum

In [ ]:
#Configurando tamanho da imagem
sns.set(rc={'figure.figsize':(11.7,8.27)})
#Fazendo um plot para análise de gastos por ano
sns.lineplot(x='year', y='total_net_value', data = year_sum)

O gasto parlamentar com a cota vem aumentando ano a ano, com uma leve queda em 2017.

http://g1.globo.com/politica/noticia/2013/03/camara-aumenta-valor-de-cota-parlamentar-e-auxilio-moradia.html

In [ ]:
agg_fun ={
    'total_net_value': 'mean'
}

quota_mean = df.groupby('subquota_description', as_index=False).agg(agg_fun)

#Ordenando os valores de forma decrescente
quota_mean.sort_values(by='total_net_value', ascending=False)


Melhorando nossos gráficos: https://www.drawingfromdata.com/how-to-rotate-axis-labels-in-seaborn-and-matplotlib

In [ ]:
chart = sns.barplot(x='subquota_description', y = 'total_net_value', data = quota_mean.sort_values(by='total_net_value', ascending=False))
chart.set_xticklabels(chart.get_xticklabels(), rotation=45, horizontalalignment='right');

A categoria que tem o maior ticket médio é a de "Aircharft renting or charter of aircraft", sendo de ~R$ 9,490.00 por ticket, que inclui compra de passagens e aluguél de aeronaves.. A que tem o menor tícket médio é a de "Taxi, toll and parking", que é utilizada para pagamento de corridas de taxi, pedágio e estacionamento.

### Análise da subquota **'Táxi, pedágio e estacionamento'**

Esta é uma Análise Descritiva Exploratória de despesas com transporte local, analisando apenas a subquota específica para resarcimento de Táxi, pedágio e estacionamento.

O objetivo é identificar valores que podem ser considerados absurdos e os deputados que mais gastam nessa subquota.

In [ ]:
df_taxi = df[df['subquota_description'] == 'Taxi, toll and parking']

df_taxi.head()

In [ ]:
df_taxi.info()

In [ ]:
#Sumarização dos dados da categoria
df_taxi['net_values'].describe()

In [ ]:
df_taxi['total_net_value'].describe()

Embora o ticket médio das restituições seja apenas **R\\$ 32**, existe pelo menos um com valor totalmente absurdo de **R\\$ 2.700** para pagamento de taxi, pedágio e estacionamento.

In [ ]:
#Plotando a distribuição de valores de net values
sns.distplot(df_taxi['net_values'])

Vamos procurar o deputado(a) com o maior valor de ticket médio:

In [ ]:
df_taxi.loc[df_taxi['net_values'].idxmax()]

O reenbolso é da **Deputada MAGDA MOFATTO** do PR, no valor de *R$ 2.700* no dia 05/09/2017. Me parece suspeito, e você ?

http://www.camara.gov.br/cota-parlamentar/sumarizado?nuDeputadoId=2437&dataInicio=9/2017&dataFim=9/2017&despesa=&nomeHospede=&nomePassageiro=&nomeFornecedor=&cnpjFornecedor=&numDocumento=&sguf=&filtroNivel1=1&filtroNivel2=2&filtroNivel3=3


Agora, vamos localizar os deputados(as) com os maiores valores de ticket

In [ ]:
df_outliers = df_taxi.nlargest(15, 'net_values')
df_outliers

In [ ]:
agg_fun ={
    'net_values': 'sum'
}

outliers_agg = df_outliers.groupby('congressperson_name', as_index=False).agg(agg_fun)

#Ordenando os valores de forma decrescente
outliers_agg.sort_values(by='net_values', ascending=False)

In [ ]:
chart = sns.barplot(x='congressperson_name', y = 'net_values', data = outliers_agg.sort_values(by='net_values', ascending=False))
chart.set_xticklabels(chart.get_xticklabels(), rotation=45, horizontalalignment='right')

Sua equipe de 3 pessoas está trabalhando na auditoria da cota parlamentar para o período de 2014 a 2018. O objetivo final da sua análise é apontar possíveis irregularidades nos reembolsos, identificar solicitações suspeitas, deputados que devem ser investigados e partidos com maior índice de irregularidades. Construa uma análise utilizando dados e justificativas plausíveis possa ser utilizada na auditoria.

Escolha **três** categorias de cota parlamentar para realizar a análise. A narrativa com os dados e pensamento analítico sobre a categoria e os valores fazem parte da avaliação, que devem responder obrigatoriamente as seguintes perguntas **para cada categoria** analisada:

* Quais solicitações de reembolso são suspeitas ?
* Das solicitações suspeitas, escolha uma para analisar mais afundo e justifique o porque devemos indiciar o parlamentar ?
* Quais parlamentares devem ser investigados (top 5)?
* Quais partidos têm a maior taxa de suspeitos (top 5) ?
* Eleja um parlamentar para ser premiado por boa conduta. Justifique sua escolha com dados. 

Escolha **um** parlamentar de sua preferência (alguém que você votou se lembrar). Analise os gastos específicos desse parlamentar e responda às seguintes perguntas:

* O parlamentar gasta a verga completa ou tem um gasto moderado ?
* Qual o perfil de gastos do parlamentar, quais categorias o parlamentar mais gasta a verba ?
* Existe algum reembolso suspeito em relação às categorias analisadas ?
* Analisando as empresas prestadoras de serviços (cnpj_cpf), existe algo suspeito ? 